In [1]:
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LassoCV, Lasso
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.ensemble import RandomForestRegressor,ExtraTreesRegressor, GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from mlxtend.regressor import StackingCVRegressor
import xgboost as xgb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import warnings
from scipy.stats import norm, skew
warnings.filterwarnings('ignore')

In [2]:
df_train = pd.read_csv('train.csv', index_col='id')
df_train.tail()

,epoch,sat_id,x,y,z,Vx,Vy,Vz,x_sim,y_sim,z_sim,Vx_sim,Vy_sim,Vz_sim
id,,,,,,,,,,,,,,
1234089,2014-01-31T22:00:22.602,599,-21721.485878,-14048.557595,5277.807430,-1.351754,3.373418,0.004995,-20717.958996,-16245.240500,5250.939232,-1.653931,3.157321,0.079069
1234090,2014-01-31T22:25:13.240,599,-23176.890569,-8712.016936,5153.371350,-0.575955,3.764450,-0.175109,-22673.444496,-11192.339393,5243.608790,-0.945328,3.603371,-0.092202
1234091,2014-01-31T22:50:03.878,599,-23363.044794,-2906.071320,4747.247386,0.351381,3.992943,-0.372198,-23461.830699,-5570.167175,4966.813869,-0.087089,3.912550,-0.281989
1234092,2014-01-31T23:14:54.515,599,-22058.020262,3074.894039,4038.853542,1.421085,3.984793,-0.578849,-22858.679929,373.249102,4396.055679,0.920162,4.021955,-0.485364
1234093,2014-01-31T23:39:45.153,599,-19076.969715,8807.476785,3024.623998,2.587973,3.641717,-0.778817,-20656.328603,6277.934683,3518.642806,2.050990,3.844021,-0.690707


In [3]:
df_train.shape

(649912, 14)

In [4]:
df_train = df_train.reset_index(drop=True)

In [5]:
t = pd.to_datetime(df_train.epoch)
df_train.epoch = (t-datetime.datetime(1970,1,1)).apply(lambda x: x.total_seconds())

In [6]:
X = df_train[['epoch', 'sat_id', 'x_sim','y_sim', 'z_sim', 'Vx_sim','Vy_sim', 'Vz_sim']]
y = df_train[['x','y','z', 'Vx','Vy', 'Vz']]

In [7]:
def listmerge(lstlst):
    all=[]
    for lst in lstlst:
        all.extend(lst)
    return all

for column in ['x_sim','y_sim', 'z_sim', 'Vx_sim','Vy_sim', 'Vz_sim']:
    col = []
    for id in X.sat_id.unique():
        lst = X[X['sat_id']==id][column]
        lst = list(lst)
        lst.pop()
        lst.insert(0, lst[0])
        col.append(lst)
    col = listmerge(col)
    X[column+'_prev'] = col

col = []
for id in X.sat_id.unique():
    lst = X[X['sat_id']==id]['epoch']
    lst = np.diff(lst)
    col.append(np.insert(lst,0,0))
col = listmerge(col)
X['delta'] = col
X.head()   

,epoch,sat_id,x_sim,y_sim,z_sim,Vx_sim,Vy_sim,Vz_sim,x_sim_prev,y_sim_prev,z_sim_prev,Vx_sim_prev,Vy_sim_prev,Vz_sim_prev,delta
0,1.388534e+09,0,-8843.131454,13138.221690,-20741.615306,-0.907527,-3.804930,-2.024133,-8843.131454,13138.221690,-20741.615306,-0.907527,-3.804930,-2.024133,0.000
1,1.388537e+09,0,-10555.500066,1649.289367,-24473.089556,-0.303704,-4.269816,-0.616468,-8843.131454,13138.221690,-20741.615306,-0.907527,-3.804930,-2.024133,2803.000
2,1.388540e+09,0,-10571.858472,-10145.939908,-24271.169776,0.274880,-4.046788,0.718768,-10555.500066,1649.289367,-24473.089556,-0.303704,-4.269816,-0.616468,2803.001
3,1.388543e+09,0,-9149.620794,-20618.200201,-20765.019094,0.712437,-3.375202,1.718306,-10571.858472,-10145.939908,-24271.169776,0.274880,-4.046788,0.718768,2803.000
4,1.388546e+09,0,-6729.358857,-28902.271436,-14992.399986,0.989382,-2.522618,2.342237,-9149.620794,-20618.200201,-20765.019094,0.712437,-3.375202,1.718306,2803.001


In [8]:
X_x = X[['epoch', 'x_sim', 'x_sim_prev', 'Vx_sim', 'Vx_sim_prev', 'delta']]
y_x = y['x']
X_y = X[['epoch', 'y_sim', 'y_sim_prev', 'Vy_sim', 'Vy_sim_prev', 'delta']]
y_y = y['y']
X_z = X[['epoch', 'z_sim', 'z_sim_prev', 'Vz_sim', 'Vz_sim_prev', 'delta']]
y_z = y['z']
y_Vx = y['Vx']
y_Vy = y['Vy']
y_Vz = y['Vz']
X = [X_x,X_y,X_z,X_x,X_y,X_z]
y = [y_x,y_y,y_z,y_Vx,y_Vy,y_Vz]

In [9]:
def smape(satellite_predicted_values, satellite_true_values): 
    return np.mean(np.abs((satellite_predicted_values - satellite_true_values) 
        / (np.abs(satellite_predicted_values) + np.abs(satellite_true_values))))

def score(satellite_predicted_values, satellite_true_values):
    return 100*(1-np.mean(smape(satellite_predicted_values, satellite_true_values)))

In [ ]:
from sklearn.model_selection import cross_validate

lasso = LassoCV()
knr = KNeighborsRegressor(metric='manhattan', n_neighbors=9, weights='distance')
randomforest = RandomForestRegressor(min_samples_leaf=200, max_depth=9, n_estimators=500)
gb = GradientBoostingRegressor(min_samples_leaf=200, max_depth=9, n_estimators=500)
stack = StackingCVRegressor(regressors=(lasso, knr, randomforest),
                                    meta_regressor=GradientBoostingRegressor(min_samples_leaf=200, max_depth=9),
                                    random_state=42)
regressors=(lasso, knr, randomforest, gb, stack)

X_train, y_train = X[1], y[1]
for i in regressors:
    cv_results = cross_validate(i, X_train, y_train, cv=5, n_jobs=-1)
    print(np.mean(cv_results['test_score']))

0.9260232805640909
0.9219995308245552


In [10]:
from sklearn.model_selection import KFold
models = []

for i in range(len(X)):
    scaler = StandardScaler()
    x = scaler.fit_transform(X[i])
    kf = KFold(n_splits=3, random_state=42, shuffle=True)
    
    lasso = Lasso()
    knr = KNeighborsRegressor(metric='manhattan', n_neighbors=3, weights='distance')
    randomforest = RandomForestRegressor(min_samples_leaf=50, max_depth=9)

    stack = StackingCVRegressor(regressors=(lasso, knr, randomforest),
                                    meta_regressor=GradientBoostingRegressor(min_samples_leaf=50, max_depth=9),
                                    random_state=42)
    
    for train_index, test_index in kf.split(x, y[i]):
        X_train, X_test = x[train_index], x[test_index]
        y_train, y_test = y[i][train_index], y[i][test_index]

        stack.fit(X_train, y_train)
        preds = stack.predict(X_train)
        print('train', score(preds, y_train))
        preds = stack.predict(X_test)
        print('test', score(preds, y_test))
        print('-'*20)
    models.append(stack)

train 98.68771595133421
test 95.66338759500536
--------------------


KeyboardInterrupt: 

# Test sample

In [ ]:
df_test = pd.read_csv('test.csv')

In [ ]:
t = pd.to_datetime(df_test.epoch)
df_test.epoch = (t-datetime.datetime(1970,1,1)).apply(lambda x: x.total_seconds())
df_test.head()

In [ ]:
for column in ['x_sim','y_sim', 'z_sim', 'Vx_sim','Vy_sim', 'Vz_sim']:
    col = []
    for id in df_test.sat_id.unique():
        lst = df_test[df_test['sat_id']==id][column]
        lst = list(lst)
        lst.pop()
        lst.insert(0, lst[0])
        col.append(lst)
    col = listmerge(col)
    df_test[column+'_prev'] = col
    
col = []
for id in df_test.sat_id.unique():
    lst = df_test[df_test['sat_id']==id]['epoch']
    lst = np.diff(lst)
    col.append(np.insert(lst,0,0))
col = listmerge(col)
df_test['delta'] = col
df_test.head()

In [ ]:
X_x = df_test[['epoch', 'x_sim', 'x_sim_prev', 'Vx_sim', 'Vx_sim_prev', 'delta']]
X_y = df_test[['epoch', 'y_sim', 'y_sim_prev', 'Vy_sim', 'Vy_sim_prev', 'delta']]
X_z = df_test[['epoch', 'z_sim', 'z_sim_prev', 'Vz_sim', 'Vz_sim_prev', 'delta']]
X = [X_x,X_y,X_z,X_x,X_y,X_z]

In [ ]:
results = [] 

for i in range(len(X)):
    scaler = StandardScaler()
    x = scaler.fit_transform(X[i])
    df = pd.DataFrame()
    
    model = models[i]
    preds = model.predict(x)
    results.append(preds)

In [ ]:
ids = df_test.id
column=['x','y', 'z', 'Vx','Vy', 'Vz']
sub = pd.DataFrame(columns=column)
for i in range(len(results)):
    sub[column[i]] = results[i]
sub['id'] = ids
sub = sub[['id','x','y', 'z', 'Vx','Vy', 'Vz']]
sub.to_csv('submission.csv',index=False)
sub.head(10)